In [2]:
########
#Base parameters
########

lA = 2
lB = 3
eA = 4
eB = 3
p = (lA^(eA)) * (lB^(eB)) - 1 

In [3]:
F.<z> = GF(p^2, modulus=x^2+1)

In [4]:
E = EllipticCurve([F(1), F(0)])

In [5]:
########
#Setting up the respective generators
########

Gen1A = (lB^(eB)) * E.gens()[0]
Gen2A = (lB^(eB)) * E.gens()[1] #l_A^(eA) torsion generators
Gen1B = (lA^(eA)) * E.gens()[0]
Gen2B = (lB^(eA)) * E.gens()[1] #l_B^(eB) torsion generators

In [6]:
########
#Public key generation. The last two input points are the other party's basis points. The first output is the image curve, the latter two are the the public basis point images.
########
def public(E,ll,e,m,n,P,Q,PP,QQ): 
    G = m*P + n*Q
    EE = E
    Pi = PP
    Qi = QQ
    
    for i in range(0,e):
            K = (ll^(e-1-i))*G
            iso = EllipticCurveIsogeny(EE,K)
            EE = iso.codomain()
            Pi = iso(Pi)
            Qi = iso(Qi)
            G = iso(G)
    
    return EE, Pi, Qi

In [7]:
########
#Private key generation. The input curve is the other party's image curve. The last two input points are the public points. 
########
def key(E,ll,e,m,n,P,Q): 
    G = m*P + n*Q
    EE = E
    
    for i in range(0,e):
        K = (ll^(e-1-i))*G
        iso = EllipticCurveIsogeny(EE,K)
        EE = iso.codomain()
        G = iso(G)
        
    return EE.j_invariant() 

In [8]:
########
#Setting up the exchange
########

In [9]:
########
#Initiator public key generation
########

def publicA(mA,nA): 
    return public(E,lA,eA,mA,nA,Gen1A,Gen2A,Gen1B,Gen2B)

In [10]:
########
#Responder public key generation
########

def publicB(mB,nB):
    return public(E,lB,eB,mB,nB,Gen1B,Gen2B,Gen1A,Gen2A)

In [11]:
########
#Initiator private key generation
########

def keyA(mA,nA,E,P,Q): #Where the outputs of the public key are E,P,Q respectively 
    return key(E,lA,eA,mA,nA,P,Q) 

In [12]:
########
#Responder private key generation
########

def keyB(mB,nB,E,P,Q):
    return key(E,lB,eB,mB,nB,P,Q) 

In [13]:
########
#Exchange testing
########

def test(E,mA,nA,mB,nB):
    pA = publicA(mA,nA)
    pB = publicB(mB,nB)
    testkeyA = keyA(mA,nA,pB[0],pB[1],pB[2])
    testkeyB = keyB(mB,nB,pA[0],pA[1],pA[2])
    if testkeyA == testkeyB:
        return "success", testkeyA
    else:
        return "something has gone wrong"

In [14]:
test(E,3,4,5,6)

('success', 125*z + 426)